In [1]:
from convokit import Corpus, download
import re
import numpy as np
import importlib

/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/.venv/lib/python3.11/site-packages/convokit/coordination/coordination.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


An error occurred: Torch not compiled with CUDA enabled


/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/.venv/lib/python3.11/site-packages/convokit/__init__.py:29: UserWarning: If you are using ConvoKit with Google Colab, incorrect versions of some packages (ex. scipy) may be imported while runtime start. To fix the issue, restart the session and run all codes again. Thank you!
  warnings.warn(


In [2]:
from model.config  import *
# from model.data import *
# from model.optimizer import *
# from utils.finetune_utils import *


In [3]:
from utils.data_processing import DataProcesser
from utils.data_processing import TextProcesser
from utils import corpus_utils as corp
import pandas as pd
import pprint as pp

# DataLoading

### Corpus Loading Dataframes

In [3]:
import os
final_filepath_kodis = os.path.join(fine_processed_dir, 'processed_kodis.csv')
raw_filepath_kodis = os.path.join(fine_raw_dir, 'KODIS-EN.csv')
corpus_kodis_filepath = os.path.join(corpus_dir, "custom")
corp.setConversationMetadata(["buyer_is_AI", "seller_is_AI", "convo_len", "provided_outcome"
    ,"s_SVI_instrumental", "s_SVI_self", "s_SVI_process", "s_SVI_relationship"])
test = DataProcesser(final_filepath_kodis)
kodis_corpus = corp.corpusBuilder(test)

Row Index in columns


0          0
1          1
2          2
3          3
4          4
        ... 
2846    2846
2847    2847
2848    2848
2849    2849
2850    2850
Name: Row_Index, Length: 2851, dtype: int64

['predictions', 'scores']
*** 0 utterances with timestamp == 0 after imputation ***


,id,speaker,conversation_id,reply_to,timestamp,text,meta.predictions,meta.scores
0,utt0_con0,Seller_0,utt0_con0,None,1699388149,Your sudden demand for a refund is unwarranted...,None,None
1,utt0_con0,Buyer_0,utt0_con0,utt0_con0,1699388150,"This was not the item I ordered, please provid...",None,None
2,utt0_con0,Seller_0,utt0_con0,utt0_con0,1699388170,How about a compromise? I can offer you a part...,None,None
3,utt0_con0,Buyer_0,utt0_con0,utt0_con0,1699388217,"No, this was not the item I ordered. I have a ...",None,None
4,utt0_con0,Seller_0,utt0_con0,utt0_con0,1699388235,"I can scrutinize your claim, but this requires...",None,None
...,...,...,...,...,...,...,...,...
39463,utt2850_con2850,Buyer_2850,utt2850_con2850,utt2850_con2850,1718879890,Thank you I appreciate that. I shall remove m...,None,None
39464,utt2850_con2850,Seller_2850,utt2850_con2850,utt2850_con2850,1718879909,"Deal. Once I see changes on your end, I'll ini...",None,None
39465,utt2850_con2850,Buyer_2850,utt2850_con2850,utt2850_con2850,1718879998,I understand - please do follow through. I ha...,None,None
39466,utt2850_con2850,Seller_2850,utt2850_con2850,utt2850_con2850,1718880018,"You have my word, I will follow through. A sna...",None,None


,id
0,Seller_0
1,Buyer_0
2,Seller_0
3,Buyer_0
4,Seller_0
...,...
39463,Buyer_2850
39464,Seller_2850
39465,Buyer_2850
39466,Seller_2850


,id,meta.buyer_is_AI,meta.seller_is_AI,meta.convo_len,meta.provided_outcome,meta.s_SVI_instrumental,meta.s_SVI_self,meta.s_SVI_process,meta.s_SVI_relationship
0,utt0_con0,False,True,16,1,NaN,NaN,NaN,NaN
1,utt1_con1,False,False,14,0,7.0,5.5,7.0,6.5
2,utt2_con2,False,True,12,0,NaN,NaN,NaN,NaN
3,utt3_con3,True,False,12,0,7.0,7.0,7.0,5.0
4,utt4_con4,True,False,14,0,4.5,5.0,4.5,5.5
...,...,...,...,...,...,...,...,...,...
2846,utt2846_con2846,False,True,16,0,NaN,NaN,NaN,NaN
2847,utt2847_con2847,False,True,10,0,NaN,NaN,NaN,NaN
2848,utt2848_con2848,False,True,15,0,NaN,NaN,NaN,NaN
2849,utt2849_con2849,False,False,11,0,5.0,6.0,6.5,5.0


conversation metadata: ['predictions', 'scores']
utterance metadata: ['buyer_is_AI', 'seller_is_AI', 'convo_len', 'provided_outcome', 's_SVI_instrumental', 's_SVI_self', 's_SVI_process', 's_SVI_relationship']


39468it [00:00, 49385.87it/s]


Utterance conversation_ids missing from convos.id: -> 0
Conversation ids in convos not used by any utterance: -> 0
Every utterance.conversation_id exists in convos.id?: -> True
Every convos.id is referred to by at least one utterance?: -> True
Unique conversation_id’s in utterance DF: -> 2851
['utt0_con0' 'utt1_con1']


### TrainTestSplit

In [ ]:
import importlib
import model.config as config
import model.data   as data
import utils.plotting_utils as plotting


def testsplit():
    convo_df= kodis_corpus.get_conversations_dataframe()
    utt_df = kodis_corpus.get_utterances_dataframe()
    X_train, X_test, y_train, y_test = createTrainTestSplit(convo_df)
    convo_df = assignSplit(convo_df, train_ids=X_train, test_ids=X_test)
    plot_fold_summary(
        [convo_df],
        split_col="meta.split",
        outcome_col="meta.provided_outcome",
        length_col="meta.convo_len",
        preferred_splits=("train","val","test")
    )
    
for h in ["none","stratified","downsampling"]:
    config.imbalance_handling = h
    config.random_seed= 42
    importlib.reload(data)
    importlib.reload(plotting)
    from model.data import createTrainTestSplit, assignSplit
    from utils.plotting_utils import plot_fold_summary

    print(f"=== Testing imbalance_handling={h} ===")
    testsplit()

### TrainValTest and TrainValSplit with K-folds >1

In [ ]:
def testsplit(h):
    imbalance_handling = h
    convo_df= kodis_corpus.get_conversations_dataframe()
    utt_df = kodis_corpus.get_utterances_dataframe()
    X_train, X_test, y_train, y_test = createTrainTestSplit(convo_df)
    convo_train = convo_df.loc[X_train]
    train_val_id_list = createTrainValSplit(convo_train)
    data_folds=[]
    for fold, pair in enumerate(train_val_id_list, start=1):
        convo_dataframe_fold = assignSplit(convo_df, train_ids=pair[0], val_ids=pair[1], test_ids= X_test)
        convo_dataframe_fold = assignSplit(convo_df, train_ids=pair[0], val_ids=pair[1], test_ids= X_test)
        data_folds.append(convo_dataframe_fold)

    plot_fold_summary_with_ai(
        data_folds,
        split_col="meta.split",
        outcome_col="meta.provided_outcome",
        length_col="meta.convo_len",
        preferred_splits=("train","val","test")
    )
               
for h in ["none","stratified", "downsampling"]:
    config.imbalance_handling = h
    config.random_seed= 205
    config.k_folds= 3
    importlib.reload(data)
    importlib.reload(plotting)
    from model.data import assignSplit, createTrainTestSplit, downsample, train_test_split, StratifiedKFold, KFold, createTrainValSplit
    from utils.plotting_utils import plot_fold_summary_with_ai
    from model.config import *
    print(f"=== Testing imbalance_handling={h} ===")
    testsplit(h)


### TrainValTest and TrainValSplit with K-folds == 1

In [ ]:
def createTrainValSplit(convo_df_train, imbalance_handling):
    convo_ids    = np.array(convo_df_train.index.tolist())
    convo_labels = convo_df_train[f"meta.{label_metadata}"].astype(int).values
    folds = []
    if k_folds > 1:
        if imbalance_handling == "stratified":
            skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=random_seed)
            for train_idx, val_idx in skf.split(convo_ids, convo_labels):
                train_ids = convo_ids[train_idx]
                val_ids   = convo_ids[val_idx]
                folds.append((train_ids, val_ids))
            return folds
        if imbalance_handling == "downsampling":
            kf = KFold(n_splits=k_folds, shuffle = True, random_state = random_seed)
            for train_idx, val_idx in kf.split(convo_ids, convo_labels):
                train_ids = convo_ids[train_idx]
                train_ids_down = downsample(convo_df_train, train_ids)
                val_ids   = convo_ids[val_idx]
                folds.append((train_ids_down, val_ids))
                before_counts = convo_df_train.loc[train_ids, f"meta.{label_metadata}"].value_counts()
                after_counts  = pd.Series(train_ids_down).map(convo_df_train[f"meta.{label_metadata}"].to_dict()).value_counts()
                print("Before:", before_counts.to_dict())
                print(" After:", after_counts.to_dict())
            return folds
        else:
            kf = KFold(n_splits=k_folds, shuffle = True, random_state = random_seed)
            for train_idx, val_idx in kf.split(convo_ids, convo_labels):
                train_ids = convo_ids[train_idx]
                val_ids   = convo_ids[val_idx]
                folds.append((train_ids, val_ids))
            return folds
    else:
        test_size = val_size  
        if imbalance_handling == "downsampling":
            train_ids, val_ids, *_ = train_test_split(convo_ids, convo_labels,test_size=test_size, random_state=random_seed)
            train_ids_down = downsample(convo_df_train, train_ids)
            folds.append((train_ids_down, val_ids))
            before_counts = convo_df_train.loc[train_ids, f"meta.{label_metadata}"].value_counts()
            after_counts  = pd.Series(train_ids_down).map(convo_df_train[f"meta.{label_metadata}"].to_dict()).value_counts()
            print("Before:", before_counts.to_dict())
            print(" After:", after_counts.to_dict())
            return folds
        if imbalance_handling == "stratified":
            train_ids, val_ids, *_ = train_test_split( convo_ids, convo_labels, test_size= test_size, stratify=convo_labels, random_state=random_seed)
            folds.append((train_ids, val_ids))
            return folds
        else:
            train_ids, val_ids, *_ = train_test_split( convo_ids, convo_labels, test_size= test_size,  random_state=random_seed)
            folds.append((train_ids, val_ids))
            return folds


def testsplit(h):
    imbalance_handling = h
    convo_df= kodis_corpus.get_conversations_dataframe()
    utt_df = kodis_corpus.get_utterances_dataframe()
    X_train, X_test, y_train, y_test = createTrainTestSplit(convo_df)
    convo_train = convo_df.loc[X_train]
    train_val_id_list = createTrainValSplit(convo_train, imbalance_handling)
    data_folds=[]
    for fold, pair in enumerate(train_val_id_list, start=1):
        convo_dataframe_fold = assignSplit(convo_df, train_ids=pair[0], val_ids=pair[1], test_ids= X_test)
        convo_dataframe_fold = assignSplit(convo_df, train_ids=pair[0], val_ids=pair[1], test_ids= X_test)
        data_folds.append(convo_dataframe_fold)

    plot_fold_summary_with_ai(
        data_folds,
        split_col="meta.split",
        outcome_col="meta.provided_outcome",
        length_col="meta.convo_len",
        preferred_splits=("train","val","test")
    )
               
for h in ["none","stratified", "downsampling"]:
    config.imbalance_handling = h
    config.random_seed= 205
    config.k_folds= 1
    importlib.reload(data)
    importlib.reload(plotting)
    from model.data import assignSplit, createTrainTestSplit, downsample, train_test_split, StratifiedKFold, KFold
    from utils.plotting_utils import plot_fold_summary_with_ai
    from model.config import *
    print(f"=== Testing imbalance_handling={h} ===")
    testsplit(h)



# Testing Filtering

In [3]:
test.filterMatches("Accept Deal")
test.filterMatches("I Walk Away")
test.filterMatches("Submit agreement")
no_accept_df= test.filterRows("message", exclude_val="Accept Deal", case_ex=True)
no_submit_df= test.filterRows("message", exclude_val="I Walk Away", case_ex=True)
sub_filtered = test.filterRows("message", exclude_val="Submitted agreement", case_ex=True)
no_last_df = test.filterRows("message", exclude_val=["Accept Deal", "I Walk Away"], case_in=True)
display(sub_filtered)
test_corpus = corp.corpusBuilder(test)

NameError: name 'test' is not defined

### Saving filtered Dataframe with no AI

In [33]:
import importlib
import utils.data_processing  as data_processing
from IPython.display import display
importlib.reload(data_processing)
from utils.data_processing import DataProcesser

#save to csv as processed data
fine_processed_filename = "kodis_no_AI_no_last.csv"
save_path_file = os.path.join(fine_processed_dir, fine_processed_filename)


corpus_data = fine_raw_file
corpus_data_no_AI = DataProcesser(corpus_data, label = True, drop_AI=True)
corpus_data_AI = DataProcesser(corpus_data, label = True)
corpus_data_no_AI.saveToCSV(save_path_file, drop_parsed= True)

corpus_data_no_AI_test = DataProcesser(save_path_file, label = True, drop_AI=True)
display(corpus_data_no_AI.getUtterancesDF())
display(corpus_data_no_AI.getDataframe())
display(corpus_data_no_AI_test.getUtterancesDF())
display(corpus_data_no_AI_test.getDataframe())
#filter utterance dataframe
# config.finetune_exclude_phrases =["I Walk Away", "Accept Deal"]
# message": ["I Walk Away", "Accept Deal"]

idx_list = corpus_data_no_AI.getDataframe().index.tolist()

#need dataframe indices to match
finetune_conversation_map = {
   'buyer_is_AI': {"include": [[False], None], "exclude": [[], None]},
   'seller_is_AI':{"include": [[False], None], "exclude": [[], None]}
}

finetune_utterance_map = {
   'message': {"include": [[], False], "exclude": [["I Walk Away", "Accept Deal"], True]},
   'speaker_id': {"include": [[], None], "exclude": [[], None]},
   'is_AI':  {"include": [[], None], "exclude": [[], None]},
}

corpus_data_no_AI.contextSelection(filter_map_convo=finetune_conversation_map, filter_map_utt=finetune_utterance_map)
display(corpus_data_no_AI.getUtterancesDF())
display(corpus_data_no_AI.getDataframe())

test_corpus = corp.corpusBuilder(corpus_data_no_AI)


Dropping 9 rows with missing formattedChat
2851
Dropping 0 rows with missing formattedChat
Dropping 0 rows with missing Outcome
Dropping 9 rows with missing formattedChat
2851
Dropping 0 rows with missing formattedChat
Dropping 0 rows with missing Outcome
Data saved to /Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/finetuning_preprocessing/processed/kodis/kodis_no_AI_no_last.csv
Row Index in columns
Dropping 0 rows with missing formattedChat
2076
Dropping 0 rows with missing formattedChat
Dropping 0 rows with missing Outcome


/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/utils/data_processing.py:132: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1702723625' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.utterancesDF.loc[13988, 'timestamp']= '1702723625'


,timestamp,message,uttidx,speaker_id,is_AI,row_idx,match_idx,Case Match Type,convo_len
0,nan,Your sudden demand for a refund is unwarranted...,0,Seller_0,True,0,False,None,16
1,1699388150,"This was not the item I ordered, please provid...",1,Buyer_0,False,0,False,None,16
2,1699388170,How about a compromise? I can offer you a part...,2,Seller_0,True,0,False,None,16
3,1699388217,"No, this was not the item I ordered. I have a ...",3,Buyer_0,False,0,False,None,16
4,1699388235,"I can scrutinize your claim, but this requires...",4,Seller_0,True,0,False,None,16
...,...,...,...,...,...,...,...,...,...
39463,1718879890,Thank you I appreciate that. I shall remove m...,5,Buyer_2859,False,2859,False,None,10
39464,1718879909,"Deal. Once I see changes on your end, I'll ini...",6,Seller_2859,True,2859,False,None,10
39465,1718879998,I understand - please do follow through. I ha...,7,Buyer_2859,False,2859,False,None,10
39466,1718880018,"You have my word, I will follow through. A sna...",8,Seller_2859,True,2859,False,None,10


,ID,b_Gender,b_Education,b_country,b_Risk,b_Refund,b_MyReview,b_OtherRev,b_Apol,b_RankBuy_1,...,b_points,joint_points,Integrative_Potential,Outcome,buyer_is_AI,seller_is_AI,convo_len,flag_speaker,dispute_outcome,provided_outcome
1,2,Female,Bachelor's Degree,United States of America,4.0,because i didn't get what I ordered I deserve ...,I want the review up on the site of the seller...,because it could potentially harm my ability t...,An updated review of me and an apology is just...,40.0,...,63.666667,125.333333,0.686587,Resolution,False,False,14,3.0,0.0,0
6,7,Male,High school diploma or GED,United States of America,5.0,This is for my nephew and you must correct thi...,Because hes a crook and will defraud others,He ripped me off and then tried to damage my r...,Because im right.,50.0,...,62.000000,92.000000,0.961846,Resolution,False,False,14,3.0,0.0,0
9,10,Female,Bachelor's Degree,United States of America,3.0,Because I did not get what I paid for and I do...,Because it was honest and it may protect other...,Because I did not do anything wrong and I shou...,Because it would correct a wrongdoing.,80.0,...,100.000000,148.333333,0.540170,Resolution,False,False,13,1.0,0.0,0
10,11,Female,High school diploma or GED,United States of America,4.0,I would want a full refund because I didn't re...,so future buyers would be aware of the risk of...,because I would want to protect my reputation,it might show other customers that the seller ...,40.0,...,66.666667,93.333333,0.966667,Resolution,False,False,12,3.0,0.0,0
12,13,Female,Bachelor's Degree,United States of America,3.0,because I could buy the shirt that I really wa...,so that I don't do it again,because he is lying and damaging my reputation,because he did something wrong and he must mak...,90.0,...,95.000000,140.000000,0.233164,Resolution,False,False,15,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,2879,Female,Bachelor's Degree,South Africa,7.0,I received the wrong package order and i belie...,i would not want other customers to have the s...,the seller was in the wrong and they should re...,it will make me feel better and valued as cust...,15.0,...,47.500000,112.500000,0.880990,Resolution,False,False,10,3.0,0.0,0
2848,2880,Female,Bachelor's Degree,Poland,1.0,I did not receive what I wanted and could use ...,So that others are aware that seller is not ho...,I would be able to buy from others as they wil...,It will show other sellers that I am not disho...,90.0,...,NaN,20.000000,0.337056,Impass,False,False,16,2.0,1.0,1
2849,2881,Male,Some college but no degree,United Kingdom of Great Britain and Northern I...,2.0,"I have not received the item I bought, so I am...",So other people don't buy from the seller and ...,Because it's untrue and could affect my standi...,For the personal satisfaction of showing the p...,75.0,...,100.000000,146.000000,0.738389,Resolution,False,False,11,1.0,0.0,0
2850,2882,Female,Bachelor's Degree,South Africa,4.0,I was not happy with what I bought,to alert others on what they might face if the...,To keep my name clean,To show that they see what they did was totall...,35.0,...,40.000000,133.333333,0.699686,Resolution,False,False,18,3.0,0.0,0


,timestamp,message,uttidx,speaker_id,is_AI,row_idx,match_idx,Case Match Type,convo_len
0,1699388451,Hi there! I believe I received the wrong jerse...,0,Buyer_1,False,1,False,None,14
1,1699388553,Good afternoon! I looked over your purchase re...,1,Seller_1,False,1,False,None,14
2,1699388719,I appreciate your timely response! I can under...,2,Buyer_1,False,1,False,None,14
3,1699388896,Of course! We try to reach all our customers a...,3,Seller_1,False,1,False,None,14
4,1699389108,"I can return the generic jersey, no problem! I...",4,Buyer_1,False,1,False,None,14
...,...,...,...,...,...,...,...,...,...
27116,1718879513,"Cool no problem, so just to clarify, you will ...",6,Buyer_2858,False,2858,False,None,11
27117,1718879641,I will refund you after you have sent the item...,7,Seller_2858,False,2858,False,None,11
27118,1718879680,"Great thanks for this outcome, enjoy the rest ...",8,Buyer_2858,False,2858,False,None,11
27119,1718879753,"Submitted agreement: Buyer gets full refund, s...",9,Seller_2858,False,2858,False,None,11


...

,id
0,Buyer_1
1,Seller_1
2,Buyer_1
3,Seller_1
4,Buyer_1
...,...
25040,Seller_2858
25041,Buyer_2858
25042,Seller_2858
25043,Buyer_2858


,id,meta.buyer_is_AI,meta.seller_is_AI,meta.convo_len,meta.provided_outcome,meta.s_SVI_instrumental,meta.s_SVI_self,meta.s_SVI_process,meta.s_SVI_relationship
0,utt1_con1,False,False,14,0,7.0,5.5,7.0,6.5
1,utt6_con6,False,False,14,0,7.0,5.5,4.0,5.0
2,utt9_con9,False,False,13,0,5.5,5.5,4.5,4.5
3,utt10_con10,False,False,12,0,NaN,NaN,NaN,NaN
4,utt12_con12,False,False,15,0,7.0,7.0,5.0,7.0
...,...,...,...,...,...,...,...,...,...
2071,utt2847_con2847,False,False,10,0,5.0,5.5,5.0,6.0
2072,utt2848_con2848,False,False,16,1,NaN,NaN,3.5,NaN
2073,utt2849_con2849,False,False,11,0,NaN,NaN,NaN,NaN
2074,utt2850_con2850,False,False,18,0,7.0,5.5,6.5,6.0


conversation metadata: ['predictions', 'scores']
utterance metadata: ['buyer_is_AI', 'seller_is_AI', 'convo_len', 'provided_outcome', 's_SVI_instrumental', 's_SVI_self', 's_SVI_process', 's_SVI_relationship']


25045it [00:00, 53688.95it/s]


Utterance conversation_ids missing from convos.id: -> 0
Conversation ids in convos not used by any utterance: -> 0
Every utterance.conversation_id exists in convos.id?: -> True
Every convos.id is referred to by at least one utterance?: -> True
Unique conversation_id’s in utterance DF: -> 2076
['utt1_con1' 'utt6_con6']


# Testing Context Creation

In [ ]:
from model.data import loadPrecomputedVoc
from model.data import tokenize
utt_label_metadata = None
label_metadata = "provided_outcome"
def processLabeledDialogs(utt_df, voc):
    utt_df = utt_df.copy()
    utt_df["tokens"] = (utt_df["text"].map(tokenize) #-> tokens for eack text
        .map(lambda toks: [t if t in voc.word2index else "UNK" for t in toks])
    )
    if utt_label_metadata is None:
        print("utt metadata empty")
        utt_df["is_attack"] = 0
    else:
        print("utt metadata exists")
        utt_df["is_attack"] = (utt_df[f"meta.{utt_label_metadata}"].fillna(0).astype(int))
    return utt_df

def loadLabeledPairs(voc,utt_df, conv_df, last_only_train, last_only_val, last_only_test):
    utts = processLabeledDialogs(utt_df, voc)
    # Prepare split IDs
    splits = {}
    for split in ("train", "val", "test"):
        ids = conv_df.index[conv_df["meta.split"] == split].unique()
        splits[split] = set(ids)
        # print(f"Split {split} has {ids} conversations")


    def make_pairs_for_split(ids_set, last_only):
        pairs = []
        for convo_id, dialog_df in utts[utts["conversation_id"].isin(ids_set)].groupby("conversation_id", sort=False):
            dialog_df['label'] = dialog_df.index
            dialog = dialog_df[["tokens", "is_attack", 'label']].to_dict("records")
            #If we only have conversation-level labels, we assume the entire dialog 
            # is a context since it does not include a derailment devent. therefore, last comment needs to be encoded
            if utt_label_metadata is None:
                if label_metadata is None:
                    raise ValueError("If utt_label_metadata is None, label_metadata must be provided to identify the conversation label.")
                    return
                last_id = dialog[-1]["label"]
                conv_label = conv_df.at[convo_id, f"meta.{label_metadata}"]
                dialog.append({"tokens": ["UNK"],"is_attack": conv_label, "label": f"{last_id}_dummyreply"
                })
            idxs = [len(dialog) - 1] if last_only else range(1, len(dialog))
            for idx in idxs:
                context = [u["tokens"][: MAX_LENGTH - 1] for u in dialog[max(0, idx - CONTEXT_SIZE) : idx]]
                reply     = dialog[idx]["tokens"][: MAX_LENGTH - 1]
                label     = conv_df.at[convo_id, f"meta.{label_metadata}"] if label_metadata else dialog[idx]["is_attack"]
                # We make a forecast on the comment prior to the reply, so we use the label of the previous comment.
                comment_id = dialog[idx - 1]["label"]
                pairs.append((context, reply, label, comment_id))
        return pairs
    train_pairs = make_pairs_for_split(splits["train"],last_only_train)
    val_pairs   = make_pairs_for_split(splits["val"],last_only_val)
    test_pairs  = make_pairs_for_split(splits["test"], last_only_test)
    return train_pairs, val_pairs, test_pairs

In [ ]:

voc = loadPrecomputedVoc(corpus_name, word2index_path, index2word_path)

In [ ]:
utts = kodis_corpus.get_utterances_dataframe().head(40)
utts["meta.provided_outcome"] = [ 1,0,1,1,1,0,2,3,4,6] + [ 1,0,1,1,1,0,2,3,4,6] +[ 1,0,1,1,1,0,2,3,4,6] +[ 1,0,1,1,1,0,2,3,4,6]
display(utts)

In [ ]:
conv_df =kodis_corpus.get_conversations_dataframe().head(5)
conv_df['meta.split'] =[ 'train', 'train', 'val', 'test', 'test']
display(conv_df)

In [ ]:
train, val, test = loadLabeledPairs(voc, utts, conv_df, False,False, False)
print(f"Train pairs: {len(train)}")
print(f"train label: {train[1][3]}")
print(f"Val pairs: {len(val)}")
print(f"Test pairs: {len(test)}")
print(val)
train, val, test = loadLabeledPairs(voc, utts, conv_df, True, True, True)
print(f"Train pairs: {len(train)}")
print(f"Val pairs: {len(val)}")
print(f"Test pairs: {len(test)}")
print(val)

# Tokenization Checks
- check the unknown tokens in the no_AI and AI disputes from WIKI vocabulary, as well as differences between the two
- find a way to handle misspellied words
- handle converting emojis

### Data prep

In [ ]:
wiki_i2w = "/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/nn_preprocessing/wikiconv/index2word.json"
wiki_w2i = "/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/nn_preprocessing/wikiconv/word2index.json"
raw_data = DataProcesser(raw_filepath_kodis)
raw_data_AI = DataProcesser(raw_filepath_kodis)
raw_data.filterValidOutcomes()
raw_data_AI.filterValidOutcomes(remove_AI=False)

In [ ]:
voc_wiki = loadPrecomputedVoc(corpus_name="wiki", word2index_path=wiki_w2i, index2word_path=wiki_i2w)
corpus_kodis_no_AI = corp.corpusBuilder(raw_data)

In [ ]:
corpus_kodis_AI = corp.corpusBuilder(raw_data_AI)

### Missing Tokens

In [ ]:
unkown_set_no_AI = set()
for utt in corpus_kodis_no_AI.iter_utterances():
    for tok in tokenize(utt.text):
        if tok not in voc_wiki.word2index:
            unkown_set_no_AI.add(tok)

known_set_no_AI = set()
for utt in corpus_kodis_no_AI.iter_utterances():
    for tok in tokenize(utt.text):
        if tok in voc_wiki.word2index:
            known_set_no_AI.add(tok)

missing_df = pd.DataFrame({"missing_tokens": list(unkown_set_no_AI)})
print("Distinct missing  :", len(unkown_set_no_AI))
print("Distinct known    :", len(known_set_no_AI ))
print("Missing/known ratio:", len(unkown_set_no_AI) / len(known_set_no_AI ))


unknown_tokens_AI = set()
for utt in corpus_kodis_AI.iter_utterances():
    for tok in tokenize(utt.text):
        if tok not in voc_wiki.word2index:
            unknown_tokens_AI.add(tok)
known_tokens_AI = set()
for utt in corpus_kodis_AI.iter_utterances():
    for tok in tokenize(utt.text):
        if tok in voc_wiki.word2index:
            known_tokens_AI.add(tok)
unknown_tokens_AI= pd.DataFrame(unknown_tokens_AI, columns={'missing_tokens': 0})
display(unknown_tokens_AI)

missing_df = pd.DataFrame({"missing_tokens": list(unknown_tokens_AI)})
print("Distinct missing  :", len(unknown_tokens_AI))
print("Distinct known    :", len(known_tokens_AI))
print("Missing/known ratio:", len(unknown_tokens_AI) / len(known_tokens_AI))

In [ ]:
diff = unknown_tokens_AI.merge(
    unknown_tokens,
    how="outer",
    on=['missing_tokens'], 
    indicator=True,
)
diff = diff.loc[diff["_merge"] != "both"]
display(diff)

# Metric Callables

### score function

In [ ]:
from sklearn.metrics import get_scorer_names, get_scorer
test = [1,0,0,1,1] 
test_p = [1,0,0,1,1] 
train =[1,1,1,1,0]
from sklearn.metrics import get_scorer_names
get_scorer_names()
acc = get_scorer("accuracy")
acc =  acc._score_func(test, train)
print(acc)
print( (np.asarray(test) == np.asarray(train)).mean())
acc_l= get_scorer("neg_log_loss")
acc_l =  acc_l._score_func(test, train)
print(acc_l)
print( (np.asarray(test) == np.asarray(train)).mean())



In [ ]:
get_scorer_names()

### Pytorch Loss

In [ ]:
import torch.nn as nn
loss_function = "BCELoss"
def get_loss(loss_function):
    candidates = [loss_function, loss_function[0].upper() + loss_function[1:]]
    for name in candidates:
        if hasattr(nn, name):
            LossClass = getattr(nn, name)
            if isinstance(LossClass, type) and issubclass(LossClass, nn.Module):
                return LossClass()
        else:
            raise ValueError(f"`{loss_function}` is not a callable or nn.Module subclass")
        
loss = get_loss(loss_function)
print(type(loss))

### Optimizer Function

In [ ]:
encoder = nn.Linear(4, 2)
context_encoder = nn.Linear(2, 2)
attack_clf = nn.Linear(2, 1)
models = [encoder, context_encoder, attack_clf]
optimizer_type = 'adam'
learning_rate = 0.005

def setOptimizer(models):
    models = nn.ModuleList([encoder, context_encoder, attack_clf])
    if optimizer_type == 'adam':
        optimizer = torch.optim.Adam(models.parameters(), lr=learning_rate)
    elif optimizer_type == 'sgd': 
        optimizer = torch.optim.SGD(models.parameters(), lr=learning_rate, momentum=0.9)
    opt_and_sched = OptimizerWithScheduler(models=models, optimizer=optimizer)
    return opt_and_sched

opt_shed = setOptimizer(models)

In [ ]:
params_from_models = list(encoder.parameters()) + list(context_encoder.parameters()) + list(attack_clf.parameters())
params_in_optimizer = opt_shed.optimizer.param_groups[0]['params']
print(set(params_in_optimizer) == set(params_from_models))

### Predictor  Activation Function

In [ ]:
import torch.nn.functional as F
from functools import partial
import inspect
def setPredictorActivation(act_name: str, **kwargs):
    name = act_name.lower()
    if not hasattr(F, name):
        raise ValueError(f"`{act_name}` is not a valid torch.nn.functional activation")
    func = getattr(F, name)
    # Wrap in partial to bind kwargs (e.g. dim for softmax)
    return partial(func, **kwargs)

sigmoid = setPredictorActivation("sigmoid")

print("Underlying function:", sigmoid.func)        # should be F.sigmoid
print("Bound keywords:    ", sigmoid.keywords)    # should be {}
# 2. Signature
print("signature:", inspect.signature(sigmoid))

### Metric Saving

In [ ]:
# Generate sample fold_epoch_metrics for 3 folds and 5 epochs
from utils.finetune_utils import average_across_folds
import pandas as pd
from IPython.display import display
num_folds = 3
num_epochs = 5

fold_epoch_metrics = {}
for i in range(num_folds):
    fold_list = []
    for epoch in range(num_epochs):
        # Simulated metrics vary by fold and epoch
        val_scores = {
            "accuracy": 0.6 + 0.05 * i + 0.02 * epoch,
            "neg_log_loss": -0.8 + -0.03 * i - 0.01 * epoch
        }
        fold_list.append({"epoch": epoch+1, "val_scores": val_scores})
    fold_epoch_metrics[f"fold_{i+1}"] = fold_list

# Generate sample fold_batch_metrics for completeness
fold_batch_metrics = {}
for i in range(num_folds):
    batch_list = []
    for epoch in range(num_epochs):
        # Single batch loss per epoch for simplicity
        batch_list.append({"epoch": epoch+1, "iteration": (epoch)*10 + (i+1), "loss": 0.2 + 0.05 * i + 0.01 * epoch})
    fold_batch_metrics[f"fold_{i+1}"] = batch_list

# Compute the averages
all_folds_list = list(fold_epoch_metrics.values())
mean_per_epoch = average_across_folds(all_folds_list)

# Prepare a DataFrame for display
df_mean = pd.DataFrame([
    {"epoch": item["epoch"], **item["mean_val_scores"]}
    for item in mean_per_epoch
])

# Display results
display(df_mean)

# Also show the structure of fold_epoch_metrics and fold_batch_metrics
print(fold_epoch_metrics)
print(fold_batch_metrics)

### Testing metric reporting format

In [47]:
import pprint
def log_fold_to_tune(epoch: int, fold_batch_metrics):
    report_dict = {}
    for fold_name, batch_list in fold_batch_metrics.items():
        for entry in batch_list:
            if entry["epoch"] != epoch:
                continue
            iteration = entry["iteration"]
            loss = entry["loss"]
            report_dict[f"{fold_name}_train_loss_{iteration}"] = loss
    
    return report_dict


def log_epoch_to_tune(epoch: int, mean_per_epochs):
    report_dict = {}
    mean_entry = mean_per_epochs[epoch]["mean_val_scores"]
    for metric_name, metric_val in mean_entry.items():
        report_dict[f"mean_val_{metric_name}"] = metric_val
    return report_dict

fold_batch_metrics = {
    "fold_1": [
        {"epoch": 1, "iteration": 0, "loss": 0.50},
        {"epoch": 1, "iteration": 1, "loss": 0.48},
        {"epoch": 1, "iteration": 2, "loss": 0.46},
        {"epoch": 2, "iteration": 0, "loss": 0.44},
        {"epoch": 2, "iteration": 1, "loss": 0.42},
    ],
    "fold_2": [
        {"epoch": 1, "iteration": 0, "loss": 0.55},
        {"epoch": 1, "iteration": 1, "loss": 0.53},
        {"epoch": 1, "iteration": 2, "loss": 0.51},
        {"epoch": 2, "iteration": 0, "loss": 0.49},
        {"epoch": 2, "iteration": 1, "loss": 0.47},
    ],
}

mean_per_epoch = [
    {"epoch": 1, "mean_val_scores": {"accuracy": 0.80, "loss": 0.49}},
    {"epoch": 2, "mean_val_scores": {"accuracy": 0.82, "loss": 0.46}},
]
pprint.pp(fold_batch_metrics)
pprint.pp(log_fold_to_tune(2, fold_batch_metrics))
pprint.pp(log_epoch_to_tune(1, mean_per_epoch))

{'fold_1': [{'epoch': 1, 'iteration': 0, 'loss': 0.5},
            {'epoch': 1, 'iteration': 1, 'loss': 0.48},
            {'epoch': 1, 'iteration': 2, 'loss': 0.46},
            {'epoch': 2, 'iteration': 0, 'loss': 0.44},
            {'epoch': 2, 'iteration': 1, 'loss': 0.42}],
 'fold_2': [{'epoch': 1, 'iteration': 0, 'loss': 0.55},
            {'epoch': 1, 'iteration': 1, 'loss': 0.53},
            {'epoch': 1, 'iteration': 2, 'loss': 0.51},
            {'epoch': 2, 'iteration': 0, 'loss': 0.49},
            {'epoch': 2, 'iteration': 1, 'loss': 0.47}]}
{'fold_1_train_loss_0': 0.44,
 'fold_1_train_loss_1': 0.42,
 'fold_2_train_loss_0': 0.49,
 'fold_2_train_loss_1': 0.47}
{'mean_val_accuracy': 0.82, 'mean_val_loss': 0.46}


### Config Saving

In [49]:
import model.config as cfg_mod
from types import ModuleType
from model.config import *

def get_config_dict():
    out = {}
    for name, val in vars(cfg_mod).items():
        if name.startswith("_"):
            continue
        if name in cfg_mod._CONFIG_BLACKLIST:
            continue
        if isinstance(val, ModuleType) or callable(val):
            continue
        if isinstance(val, (int, float, bool, str, list, dict, tuple)):
            try:
                out[name] = val
            except (TypeError, ValueError):
                out[name] = str(val)
    return out

get_config_dict()

{'corpus_name': 'wikiconv',
 'label_metadata': 'provided_outcome',
 'label_map': {0: 'success', 1: 'impasse'},
 'finetune_corpus_name': 'kodis',
 'finetune_data': 'KODIS-EN.csv',
 'pretrained_model': 'model.tar',
 'pretrain_model_path': '/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/saved_models/wikiconv/pretrained_models/model.tar',
 'fine_raw_file': '/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/finetuning_preprocessing/raw/kodis/KODIS-EN.csv',
 'fine_processed_filename': 'Kodis_no_last_no_AI.csv',
 'fine_processed_file': '/Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/finetuning_preprocessing/processed/kodis/change_to_custom.csv',
 'pretrain_exclude_phrases': [],
 'pretrain_case': False,
 'pretrain_include_AI': True,
 'pretrain_utterance_metadata': [],
 'pretrain_convo_metadata': [],
 'finetune_conversation_map': {'buyer_is_AI': {'include': [[False], None],
   'exclude': [[], None]},
  'seller_is_AI': {'include': [

# Testing final finetune loop

### Create KODIS small test set

In [ ]:
import utils as utils
importlib.reload(config)
config.finetune_data = "KODIS-EN.csv"
config.fine_raw_file = os.path.join(fine_raw_dir, "KODIS-EN.csv")
importlib.reload(utils)
from model.config import *
print(config.fine_raw_file)
kodis_data = DataProcesser(fine_raw_file)
display(kodis_data.getDataframe())
small_convo = kodis_data.getDataframe()
small_utt = kodis_data.getUtterancesDF()
display(small_utt)
# get the first 40 by position, not by label
small_convo = kodis_data.getDataframe().iloc[:500].copy()
# then keep only matching utterances
small_utt = kodis_data.getUtterancesDF()
mask = kodis_data.getUtterancesDF()["row_idx"].isin(small_convo.index)
small_utt = kodis_data.getUtterancesDF()[mask]
# and force it back to int just in case
# small_utt["row_idx"] = small_utt["row_idx"].astype(int)
kodis_data.df = small_convo
kodis_data.utterancesDF = small_utt
final_test_file = os.path.join(fine_raw_dir, "KODIS_test.csv")
kodis_data.saveToCSV(final_test_file)
kodis_data_test = DataProcesser(final_test_file)
display(kodis_data_test.getDataframe())
display(kodis_data_test.getUtterancesDF())
print(fine_raw_dir)

### Testing MLFlow Only

In [46]:
import importlib
import os
import model.config as config
import model.data as data 
import model.model as model
import model.classifiers as classifiers
import model.optimizer as optimizer
import utils.data_processing  as dataprocesssing
import utils.corpus_utils as corpus_utils
import utils.file_utils as file_utils
import utils.plotting_utils as plotting
import utils.finetune_utils as finetune
import runners.raytune as raytune
import utils as utils




for h in ["downsampling"]:
    """
    Set up config for one fold, reload modules that consume config,
    build the config dict, and call finetune.
    """
    # 1) Reload config so you start fresh
    importlib.reload(config)
    # 2) Mutate
    config.experiment_name    = "test_1fold_ALL_KODIS_downsampling_no_scheduling"
    config.experiment_dir     = os.path.join(config.experiments_dir, config.experiment_name)
    config.finetune_utterance_map['message']['exclude'][0] = ["I Walk Away", "Accept Deal"]
    config.ray_tune           = False
    config.k_folds            = 1
    config.corpus_name        = "wikiconv"
    config.scheduling          = False
    config.save_dir_pretrain  = os.path.join(repo_dir, "saved_models", config.corpus_name, "pretrained_models")
    config.save_dir_finetune = os.path.join(repo_dir, "saved_models", config.corpus_name, "finetuned_models")
    config.pretrain_model_path = os.path.join(config.save_dir_pretrain, pretrained_model)
    config.pretrained_model_path = os.path.join(config.pretrain_model_path, "KODIS-EN.csv")
    config.word2index_path = os.path.join(data_dir, "nn_preprocessing", config.corpus_name, "word2index.json") # These two files jointly define the
    config.index2word_path = os.path.join(data_dir, "nn_preprocessing", config.corpus_name, "index2word.json")
    config.batch_size         = 32
    config.imbalance_handling = h
    config.fine_raw_file      = "KODIS-EN.csv"
    config.fine_raw_file      = os.path.join(config.fine_raw_dir, "KODIS-EN.csv")
    config.finetune_epochs    = 7
    config.fine_processed_filename = "Kodis_no_last_no_AI.csv"
    # config.finetune_processed_file = os.path.join(config.fine_processed_dir, "Kodis_no_last_no_AI.csv")

    # 3) Reload any modules that import config at top‐level
    from model.config import *
    importlib.reload(data)
    importlib.reload(model)
    importlib.reload(classifiers)
    importlib.reload(optimizer)
    importlib.reload(dataprocesssing)
    importlib.reload(corpus_utils)
    importlib.reload(file_utils)
    importlib.reload(utils)
    importlib.reload(raytune)
    importlib.reload(finetune)
    importlib.reload(plotting)
    # from model.config import *
     

    # 4) Build config dict & run
    cfg_dict = raytune.get_config_dict()
    print(f"\n=== Running with Imbalance_handling = '{config.imbalance_handling}' ===")
    print("Using file in config:", config.fine_raw_file)
    print("Using processed file in config:", config.fine_processed_filename)
    print("Using raw file in finetun utils:", utils.finetune_utils.fine_raw_file)
    utils.finetune_utils.finetune_craft(cfg_dict)


# Now call it for each mode:



=== Running with Imbalance_handling = 'downsampling' ===
Using file in config: /Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/finetuning_preprocessing/raw/kodis/KODIS-EN.csv
Using processed file in config: Kodis_no_last_no_AI.csv
Using raw file in finetun utils: /Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/finetuning_preprocessing/raw/kodis/KODIS-EN.csv
Loading data artifacts...
getting data from /Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/finetuning_preprocessing/raw/kodis/KODIS-EN.csv
dropping conversation values: {'buyer_is_AI': {'include': [[False], None], 'exclude': [[], None]}, 'seller_is_AI': {'include': [[False], None], 'exclude': [[], None]}}
dropping utterance values: {'message': {'include': [[], False], 'exclude': [['I Walk Away', 'Accept Deal'], True]}, 'speaker_id': {'include': [[], None], 'exclude': [[], None]}, 'is_AI': {'include': [[], None], 'exclude': [[], None]}}
including AI?: False
Droppi

,ID,b_Gender,b_Education,b_country,b_Risk,b_Refund,b_MyReview,b_OtherRev,b_Apol,b_RankBuy_1,...,b_points,joint_points,Integrative_Potential,Outcome,buyer_is_AI,seller_is_AI,convo_len,flag_speaker,dispute_outcome,provided_outcome
1,2,Female,Bachelor's Degree,United States of America,4.0,because i didn't get what I ordered I deserve ...,I want the review up on the site of the seller...,because it could potentially harm my ability t...,An updated review of me and an apology is just...,40.0,...,63.666667,125.333333,0.686587,Resolution,False,False,14,3.0,0.0,0
6,7,Male,High school diploma or GED,United States of America,5.0,This is for my nephew and you must correct thi...,Because hes a crook and will defraud others,He ripped me off and then tried to damage my r...,Because im right.,50.0,...,62.000000,92.000000,0.961846,Resolution,False,False,14,3.0,0.0,0
9,10,Female,Bachelor's Degree,United States of America,3.0,Because I did not get what I paid for and I do...,Because it was honest and it may protect other...,Because I did not do anything wrong and I shou...,Because it would correct a wrongdoing.,80.0,...,100.000000,148.333333,0.540170,Resolution,False,False,13,1.0,0.0,0
10,11,Female,High school diploma or GED,United States of America,4.0,I would want a full refund because I didn't re...,so future buyers would be aware of the risk of...,because I would want to protect my reputation,it might show other customers that the seller ...,40.0,...,66.666667,93.333333,0.966667,Resolution,False,False,12,3.0,0.0,0
12,13,Female,Bachelor's Degree,United States of America,3.0,because I could buy the shirt that I really wa...,so that I don't do it again,because he is lying and damaging my reputation,because he did something wrong and he must mak...,90.0,...,95.000000,140.000000,0.233164,Resolution,False,False,15,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2847,2879,Female,Bachelor's Degree,South Africa,7.0,I received the wrong package order and i belie...,i would not want other customers to have the s...,the seller was in the wrong and they should re...,it will make me feel better and valued as cust...,15.0,...,47.500000,112.500000,0.880990,Resolution,False,False,10,3.0,0.0,0
2848,2880,Female,Bachelor's Degree,Poland,1.0,I did not receive what I wanted and could use ...,So that others are aware that seller is not ho...,I would be able to buy from others as they wil...,It will show other sellers that I am not disho...,90.0,...,NaN,20.000000,0.337056,Impass,False,False,16,2.0,1.0,1
2849,2881,Male,Some college but no degree,United Kingdom of Great Britain and Northern I...,2.0,"I have not received the item I bought, so I am...",So other people don't buy from the seller and ...,Because it's untrue and could affect my standi...,For the personal satisfaction of showing the p...,75.0,...,100.000000,146.000000,0.738389,Resolution,False,False,11,1.0,0.0,0
2850,2882,Female,Bachelor's Degree,South Africa,4.0,I was not happy with what I bought,to alert others on what they might face if the...,To keep my name clean,To show that they see what they did was totall...,35.0,...,40.000000,133.333333,0.699686,Resolution,False,False,18,3.0,0.0,0


displaying utterance dataframe:


,timestamp,message,uttidx,speaker_id,is_AI,row_idx,match_idx,Case Match Type,convo_len
16,1699388451,Hi there! I believe I received the wrong jerse...,0,Buyer_1,False,1,False,None,14
17,1699388553,Good afternoon! I looked over your purchase re...,1,Seller_1,False,1,False,None,14
18,1699388719,I appreciate your timely response! I can under...,2,Buyer_1,False,1,False,None,14
19,1699388896,Of course! We try to reach all our customers a...,3,Seller_1,False,1,False,None,14
20,1699389108,"I can return the generic jersey, no problem! I...",4,Buyer_1,False,1,False,None,14
...,...,...,...,...,...,...,...,...,...
39452,1718879412,No problem. It happens sometimes. It will be a...,5,Seller_2858,False,2858,False,None,11
39453,1718879513,"Cool no problem, so just to clarify, you will ...",6,Buyer_2858,False,2858,False,None,11
39454,1718879641,I will refund you after you have sent the item...,7,Seller_2858,False,2858,False,None,11
39455,1718879680,"Great thanks for this outcome, enjoy the rest ...",8,Buyer_2858,False,2858,False,None,11


['predictions', 'scores']
*** 0 utterances with timestamp == 0 after imputation ***


...

,id
0,Buyer_1
1,Seller_1
2,Buyer_1
3,Seller_1
4,Buyer_1
...,...
25040,Seller_2858
25041,Buyer_2858
25042,Seller_2858
25043,Buyer_2858


,id,meta.buyer_is_AI,meta.seller_is_AI,meta.convo_len,meta.provided_outcome,meta.s_SVI_instrumental,meta.s_SVI_self,meta.s_SVI_process,meta.s_SVI_relationship
0,utt1_con1,False,False,14,0,7.0,5.5,7.0,6.5
1,utt6_con6,False,False,14,0,7.0,5.5,4.0,5.0
2,utt9_con9,False,False,13,0,5.5,5.5,4.5,4.5
3,utt10_con10,False,False,12,0,NaN,NaN,NaN,NaN
4,utt12_con12,False,False,15,0,7.0,7.0,5.0,7.0
...,...,...,...,...,...,...,...,...,...
2071,utt2847_con2847,False,False,10,0,5.0,5.5,5.0,6.0
2072,utt2848_con2848,False,False,16,1,NaN,NaN,3.5,NaN
2073,utt2849_con2849,False,False,11,0,NaN,NaN,NaN,NaN
2074,utt2850_con2850,False,False,18,0,7.0,5.5,6.5,6.0


conversation metadata: ['predictions', 'scores']
utterance metadata: ['buyer_is_AI', 'seller_is_AI', 'convo_len', 'provided_outcome', 's_SVI_instrumental', 's_SVI_self', 's_SVI_process', 's_SVI_relationship']


25045it [00:00, 54076.74it/s]


Utterance conversation_ids missing from convos.id: -> 0
Conversation ids in convos not used by any utterance: -> 0
Every utterance.conversation_id exists in convos.id?: -> True
Every convos.id is referred to by at least one utterance?: -> True
Unique conversation_id’s in utterance DF: -> 2076
['utt1_con1' 'utt6_con6']
imbalance handling is:downsampling
=== Loading fold artifacts for fold 1 ===
Loading model artifacts...
Loading model on CPU
loading pretrained model from /Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/saved_models/wikiconv/pretrained_models/model.tar
Loading vocab from /Users/mishkin/Desktop/Research/CRAFT_Disputes/CRAFT_Disputes/src/data/nn_preprocessing/wikiconv/word2index.json
Loading adam optimizer  with scheduler setting: False
Loading fold directories
experiment name is: test_1fold_ALL_KODIS_downsampling_no_scheduling
Loading train/val pairs
the number of train pairs is:452
computed n_iter_per epoch is: 15
starting training epoch 1...
training it

In [ ]:
from sklearn import metrics 
metrics.get_scorer_names()

In [2]:
import model.config as config
print(config.fine_raw_dir)
print(config.repo_dir)

/private/tmp/ray/session_2025-07-03_00-57-14_531798_2571/runtime_resources/working_dir_files/_ray_pkg_afe092f89cbdaa6a/src/data/finetuning_preprocessing/raw/kodis
/private/tmp/ray/session_2025-07-03_00-57-14_531798_2571/runtime_resources/working_dir_files/_ray_pkg_afe092f89cbdaa6a/src


In [ ]:
{'fold_1': [[{'epoch': 1, 'iteration': 1, 'loss': 0.6946957111358643}, {'epoch': 1, 'iteration': 11, 'loss': 0.47562432289123535}]], 'fold_2': [[{'epoch': 1, 'iteration': 1, 'loss': 0.6759616732597351}, {'epoch': 1, 'iteration': 11, 'loss': 0.47568053007125854}]], 'fold_3': [[{'epoch': 1, 'iteration': 1, 'loss': 0.7086681127548218}, {'epoch': 1, 'iteration': 11, 'loss': 0.5384546518325806}]]}